In [10]:
!pip install transformers

In [11]:
import requests
from bs4 import BeautifulSoup
import csv
from transformers import pipeline

In [12]:
# Функция для получения текста отзыва из URL
def get_review_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    review_text_element = soup.find(class_='Comment_reviewText__PEkHn')
    if review_text_element:
        review_text = review_text_element.get_text()
        return review_text
    else:
        return "Review text not found"

In [13]:
# URL страницы с отзывами
base_url = 'https://www.litres.ru/book/lev-tolstoy/voyna-i-mir-tom-1-25452548/otzivi/'

In [14]:
# Получаем количество страниц с отзывами
response = requests.get(base_url)
soup = BeautifulSoup(response.text, 'html.parser')

last_page_url_element = soup.find(class_='b-pagination__item--last')
if last_page_url_element:
    last_page_url = last_page_url_element.find('a')['href']
    last_page_number = int(last_page_url.split('=')[-1])
else:
    last_page_number = 1

In [15]:
# Инициализация модели для анализа тональности
classifier = pipeline('sentiment-analysis', model='seara/rubert-tiny2-russian-sentiment')

In [16]:
# Открываем CSV файл для записи результатов
with open('sentiment_analysis_results.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Отзыв', 'Эмоциональная окраска', 'Вероятность'])

    # Проходим по каждой странице с отзывами
    for page in range(1, last_page_number + 1):
        page_url = f'{base_url}?page={page}'
        response = requests.get(page_url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Получаем тексты отзывов на текущей странице
        review_elements = soup.find_all(class_='Comment_reviewText__PEkHn')

        for review_element in review_elements:
            review_text = review_element.get_text()

            # Анализируем текст отзыва
            predictions = classifier([review_text])

            # Записываем результаты анализа в CSV файл
            for prediction in predictions:
                writer.writerow([review_text, prediction['label'], f'{prediction["score"]:.4f}'])

print("Результаты сохранены в файл sentiment_analysis_results.csv")

Результаты сохранены в файл sentiment_analysis_results.csv
